## Export SynPUF CSV Dataset to SQLite DB

This notebook loads SynPUF dataset in CSV format and export it to SQLite DB. It creates a database with four tables:
* beneficiary_summary
* inpatient_claims
* outpatient_claims
* prescription_drug_events

In [1]:
import pandas as pd
import os

In [2]:
# DATA_ROOT_DIR = '../data/sample_1/'

DATA_ROOT_DIR = 's3://aws-gaiic-merck-text2sql/data/sample_1/'
DATABASE_PATH = '../database/synpuf.db'

TABLES_INFO = {
    "beneficiary_summary": [
        "DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv",
        "DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv",
        "DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv"
    ],
    "inpatient_claims": [
        "DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv"
    ],
    "outpatient_claims": [
        "DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv"
    ],
    "prescription_drug_events": [
        "DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv"
    ]
}

### Load CSV Data

In [3]:
def load_data(table_name, data_dir=DATA_ROOT_DIR, tables_info=TABLES_INFO):
    """Load data from csv files for a specific table name."""
    fnames = tables_info[table_name]
    df = pd.DataFrame()
    for fname in fnames:
        d_path = os.path.join(data_dir, fname)
        df0 = pd.read_csv(d_path, low_memory=False)
        if not len(df):
            df = df0.copy()
        else:
            df = pd.concat([df, df0], axis=0)
            
    #Convert date columns to datetime type
    dt_cols = [col for col in df.columns if col.endswith('_DT')]
    for col in dt_cols:
        df[col] = pd.to_datetime(df[col], format='%Y%m%d')

    return df

In [4]:
df_beneficiary_summary = load_data(table_name="beneficiary_summary")
print(df_beneficiary_summary.shape)
df_beneficiary_summary.head()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


(343644, 32)


DESYNPUF_ID BENE_BIRTH_DT BENE_DEATH_DT  BENE_SEX_IDENT_CD  \
0  00013D2EFD8E45D1    1923-05-01           NaT                  1   
1  00016F745862898F    1943-01-01           NaT                  1   
2  0001FDD721E223DC    1936-09-01           NaT                  2   
3  00021CA6FF03E670    1941-06-01           NaT                  1   
4  00024B3D2352D2D0    1936-08-01           NaT                  1   

   BENE_RACE_CD BENE_ESRD_IND  SP_STATE_CODE  BENE_COUNTY_CD  \
0             1             0             26             950   
1             1             0             39             230   
2             1             0             39             280   
3             5             0              6             290   
4             1             0             52             590   

   BENE_HI_CVRAGE_TOT_MONS  BENE_SMI_CVRAGE_TOT_MONS  ...  SP_STRKETIA  \
0                       12                        12  ...            2   
1                       12                        12  ...            2   
2                       12                        12  ...            2   
3                        0                         0  ...            2   
4                       12                        12  ...            2   

   MEDREIMB_IP  BENRES_IP  PPPYMT_IP  MEDREIMB_OP  BENRES_OP  PPPYMT_OP  \
0          0.0        0.0        0.0         50.0       10.0        0.0   
1          0.0        0.0        0.0          0.0        0.0        0.0   
2          0.0        0.0        0.0          0.0        0.0        0.0   
3          0.0        0.0        0.0          0.0        0.0        0.0   
4          0.0        0.0        0.0         30.0       40.0        0.0   

   MEDREIMB_CAR  BENRES_CAR  PPPYMT_CAR  
0           0.0         0.0         0.0  
1         700.0       240.0         0.0  
2           0.0         0.0         0.0  
3           0.0         0.0         0.0  
4         220.0        80.0         0.0  

[5 rows x 32 columns]

In [5]:
df_inpatient_claims = load_data(table_name="inpatient_claims")
print(df_inpatient_claims.shape)
df_inpatient_claims.head()

(66773, 81)


DESYNPUF_ID           CLM_ID  SEGMENT CLM_FROM_DT CLM_THRU_DT  \
0  00013D2EFD8E45D1  196661176988405        1  2010-03-12  2010-03-13   
1  00016F745862898F  196201177000368        1  2009-04-12  2009-04-18   
2  00016F745862898F  196661177015632        1  2009-08-31  2009-09-02   
3  00016F745862898F  196091176981058        1  2009-09-17  2009-09-20   
4  00016F745862898F  196261176983265        1  2010-06-26  2010-07-01   

  PRVDR_NUM  CLM_PMT_AMT  NCH_PRMRY_PYR_CLM_PD_AMT  AT_PHYSN_NPI  \
0    2600GD       4000.0                       0.0  3.139084e+09   
1    3900MB      26000.0                       0.0  6.476809e+09   
2    3900HM       5000.0                       0.0  6.119985e+08   
3    3913XU       5000.0                       0.0  4.971603e+09   
4    3900MB      16000.0                       0.0  6.408400e+09   

   OP_PHYSN_NPI  ...  HCPCS_CD_36 HCPCS_CD_37 HCPCS_CD_38  HCPCS_CD_39  \
0           NaN  ...          NaN         NaN         NaN          NaN   
1           NaN  ...          NaN         NaN         NaN          NaN   
2  6.119985e+08  ...          NaN         NaN         NaN          NaN   
3           NaN  ...          NaN         NaN         NaN          NaN   
4  1.960860e+09  ...          NaN         NaN         NaN          NaN   

   HCPCS_CD_40  HCPCS_CD_41  HCPCS_CD_42  HCPCS_CD_43 HCPCS_CD_44 HCPCS_CD_45  
0          NaN          NaN          NaN          NaN         NaN         NaN  
1          NaN          NaN          NaN          NaN         NaN         NaN  
2          NaN          NaN          NaN          NaN         NaN         NaN  
3          NaN          NaN          NaN          NaN         NaN         NaN  
4          NaN          NaN          NaN          NaN         NaN         NaN  

[5 rows x 81 columns]

In [6]:
df_outpatient_claims = load_data(table_name="outpatient_claims")
print(df_outpatient_claims.shape)
df_outpatient_claims.head()

(790790, 76)


DESYNPUF_ID           CLM_ID  SEGMENT CLM_FROM_DT CLM_THRU_DT  \
0  00013D2EFD8E45D1  542192281063886        1  2008-09-04  2008-09-04   
1  00016F745862898F  542272281166593        1  2009-06-02  2009-06-02   
2  00016F745862898F  542282281644416        1  2009-06-23  2009-06-23   
3  0001FDD721E223DC  542642281250669        1  2009-10-11  2009-10-11   
4  00024B3D2352D2D0  542242281386963        1  2008-07-12  2008-07-12   

  PRVDR_NUM  CLM_PMT_AMT  NCH_PRMRY_PYR_CLM_PD_AMT  AT_PHYSN_NPI  \
0    2600RA         50.0                       0.0  4.824842e+09   
1    3901GS         30.0                       0.0  2.963420e+09   
2    3939PG         30.0                       0.0  5.737808e+09   
3    3902NU         30.0                       0.0  1.233848e+09   
4    5200TV         30.0                       0.0  9.688809e+09   

   OP_PHYSN_NPI  ...  HCPCS_CD_36  HCPCS_CD_37 HCPCS_CD_38 HCPCS_CD_39  \
0           NaN  ...          NaN          NaN         NaN         NaN   
1           NaN  ...          NaN          NaN         NaN         NaN   
2           NaN  ...          NaN          NaN         NaN         NaN   
3           NaN  ...          NaN          NaN         NaN         NaN   
4           NaN  ...          NaN          NaN         NaN         NaN   

  HCPCS_CD_40 HCPCS_CD_41 HCPCS_CD_42 HCPCS_CD_43 HCPCS_CD_44 HCPCS_CD_45  
0         NaN         NaN         NaN         NaN         NaN         NaN  
1         NaN         NaN         NaN         NaN         NaN         NaN  
2         NaN         NaN         NaN         NaN         NaN         NaN  
3         NaN         NaN         NaN         NaN         NaN         NaN  
4         NaN         NaN         NaN         NaN         NaN         NaN  

[5 rows x 76 columns]

In [ ]:
df_drug_events = load_data(table_name="prescription_drug_events")
print(df_drug_events.shape)
df_drug_events.head()

In [7]:
# output path for the CSV files
OUTPUT_DIR = 's3://aws-gaiic-merck-text2sql/data/sample_1_processed/'

# df_beneficiary_summary.dropna(axis=1, how='all', inplace=True)
# output_path = os.path.join(OUTPUT_DIR, 'beneficiary_summary.csv')
# df_beneficiary_summary.to_csv(output_path, index=False)

# df_inpatient_claims.dropna(axis=1, how='all', inplace=True)
# output_path = os.path.join(OUTPUT_DIR, 'inpatient_claims.csv')
# df_inpatient_claims.to_csv(output_path, index=False)

# df_outpatient_claims.dropna(axis=1, how='all', inplace=True)
# output_path = os.path.join(OUTPUT_DIR, 'outpatient_claims.csv')
# df_outpatient_claims.to_csv(output_path, index=False)

df_drug_events.dropna(axis=1, how='all', inplace=True)
output_path = os.path.join(OUTPUT_DIR, 'prescription_drug_events.csv')
df_drug_events.to_csv(output_path, index=False)

In [25]:
# excluded_columns = [col for col in df_inpatient_claims.columns if col.startswith("HCPCS_CD")]
# len(excluded_columns), len(df_inpatient_claims.columns)-len(excluded_columns)

(45, 36)

In [23]:
df_inpatient_claims.dropna(axis=1, how='all', inplace=True)

In [24]:
df0.columns

Index(['DESYNPUF_ID', 'CLM_ID', 'SEGMENT', 'CLM_FROM_DT', 'CLM_THRU_DT',
       'PRVDR_NUM', 'CLM_PMT_AMT', 'NCH_PRMRY_PYR_CLM_PD_AMT', 'AT_PHYSN_NPI',
       'OP_PHYSN_NPI', 'OT_PHYSN_NPI', 'CLM_ADMSN_DT', 'ADMTNG_ICD9_DGNS_CD',
       'CLM_PASS_THRU_PER_DIEM_AMT', 'NCH_BENE_IP_DDCTBL_AMT',
       'NCH_BENE_PTA_COINSRNC_LBLTY_AM', 'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM',
       'CLM_UTLZTN_DAY_CNT', 'NCH_BENE_DSCHRG_DT', 'CLM_DRG_CD',
       'ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4',
       'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6', 'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8',
       'ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10', 'ICD9_PRCDR_CD_1',
       'ICD9_PRCDR_CD_2', 'ICD9_PRCDR_CD_3', 'ICD9_PRCDR_CD_4',
       'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6'],
      dtype='object')

### Create SQLite DB and Export Data

In [41]:
import sqlite3
con = sqlite3.connect(DATABASE_PATH)

In [44]:
df_beneficiary_summary.to_sql(name="beneficiary_summary", index=False, con=con)

343644

In [45]:
df_inpatient_claims.to_sql(name="inpatient_claims", index=False, con=con)

66773

In [46]:
df_outpatient_claims.to_sql(name="outpatient_claims", index=False, con=con)

790790

In [47]:
df_drug_events.to_sql(name="prescription_drug_events", index=False, con=con)

5552421

In [ ]:
con.close()

### Create DB Schema

In [21]:
DATABASE_PATH = '../database/synpuf.db'
SCHEMA_PATH = '../database/synpuf_schema.txt'

In [6]:
import sqlite3
con = sqlite3.connect(DATABASE_PATH)

In [17]:
def get_db_schema(db_path):
    "Get database schema from the given path."
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    db_schema = []
    for table_name in tables:
        table_name = table_name[0]
        stmt = f"SELECT sql FROM sqlite_schema WHERE name = '{table_name}';"
        schema = cursor.execute(stmt)
        schema = schema.fetchall()
        db_schema.append(schema[0][0])
    cursor.close()
    conn.close()

    db_schema = "\n\n".join(db_schema)
    return db_schema

In [19]:
db_schema = get_db_schema(DATABASE_PATH)

In [24]:
print(db_schema[:100])

CREATE TABLE "inpatient_claims" (
"DESYNPUF_ID" TEXT,
  "CLM_ID" TEXT,
  "SEGMENT" TEXT,
  "CLM_FROM


In [25]:
with open(SCHEMA_PATH, 'w') as fp:
    fp.write(db_schema)

## Draft

In [31]:
con = sqlite3.connect(DATABASE_PATH)

In [80]:
# sql = "SELECT desynpuf_id FROM inpatient_claims where datediff(year, bene_birth_dt, bene_death_dt) <= 30"

def get_datetime_diff(datetime1, datetime2, unit, column_alias=None):
    """Gets time difference between two dates."""
    
    diff = f'(JULIANDAY({datetime2}) - JULIANDAY({datetime1}))'
    if unit=='year':
        diff += '/365.0'
    elif unit == 'second':
        diff += '*86400.0'
    elif unit == 'day':
        diff += ''
    else:
        raise ValueError('Invalid unit time! Allowed are year/day/second')
        
    if column_alias is None:
        diff = f'({diff})'
    else:
        diff = f'({diff}) AS {column_alias}'
        
    
    return diff

diff = get_datetime_diff(datetime1='bene_birth_dt', 
                         datetime2='bene_death_dt', 
                         unit='year')
sql = f"SELECT desynpuf_id FROM beneficiary_summary WHERE {diff} <= 30;"
print(sql)

SELECT desynpuf_id FROM beneficiary_summary WHERE ((JULIANDAY(bene_death_dt) - JULIANDAY(bene_birth_dt))/365.0) <= 30;


In [ ]:
(julianday(bene_death_dt) - julianday(bene_birth_dt))

In [69]:
df0 = pd.read_sql(sql, con=con)

In [70]:
df0.head()

DESYNPUF_ID BENE_DEATH_DT        BENE_BIRTH_DT  AGE
0  00013D2EFD8E45D1          None  1923-05-01 00:00:00  NaN
1  00016F745862898F          None  1943-01-01 00:00:00  NaN
2  0001FDD721E223DC          None  1936-09-01 00:00:00  NaN
3  00021CA6FF03E670          None  1941-06-01 00:00:00  NaN
4  00024B3D2352D2D0          None  1936-08-01 00:00:00  NaN

In [71]:
df0[~df0.AGE.isna()]

DESYNPUF_ID        BENE_DEATH_DT        BENE_BIRTH_DT      AGE
118     0033C90574583C0D  2008-09-01 00:00:00  1936-08-01 00:00:00  26329.0
165     00521D781532612C  2008-07-01 00:00:00  1937-08-01 00:00:00  25902.0
200     00653DC86A0E30D8  2008-05-01 00:00:00  1949-06-01 00:00:00  21519.0
214     006D438FFACFB5FD  2008-01-01 00:00:00  1920-06-01 00:00:00  31990.0
253     0083EA0267C0F7CA  2008-02-01 00:00:00  1935-12-01 00:00:00  26360.0
...                  ...                  ...                  ...      ...
343248  FF094311E9B48B36  2010-03-01 00:00:00  1941-05-01 00:00:00  25141.0
343256  FF0EBDC69346DD31  2010-12-01 00:00:00  1930-06-01 00:00:00  29403.0
343429  FF76CFAEBE33E0E6  2010-01-01 00:00:00  1962-12-01 00:00:00  17198.0
343489  FFA7DE17270D276C  2010-04-01 00:00:00  1931-05-01 00:00:00  28825.0
343640  FFFE3ED9582AD46E  2010-11-01 00:00:00  1916-06-01 00:00:00  34486.0

[5461 rows x 4 columns]